In [1]:
from model.model import ConvModel, SimCLR
import argparse
import torch
import torch.backends.cudnn as cudnn
from torchvision import models
from torchvision import transforms, datasets


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
    gpu_index = 0
else:
    device = torch.device('cpu')
    gpu_index = -1


In [5]:
composed_transforms = transforms.Compose([
        transforms.GaussianBlur(kernel_size=3),
      transforms.RandomRotation(degrees=10), # TODO
      transforms.RandomGrayscale(p=0.2),
      transforms.ToTensor()])

In [6]:
train_dataset = datasets.MNIST('./datasets', train=True,
                               transform=composed_transforms,
                               download=True)
val_dataset = datasets.MNIST('./datasets', download=True, transform=transforms.ToTensor())
g = torch.Generator()
g.manual_seed(0)

In [7]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=256, shuffle=False,
     pin_memory=True, drop_last=True, generator=g)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=256, shuffle=True)

In [8]:
model = ConvModel(out_dim=10)

# model.load_state_dict(torch.load('saved_models/model_01_no_aug_random_labels.pth'))


C:\Users\kacpe\DataspellProjects\SimCLRPytorch\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\kacpe\DataspellProjects\SimCLRPytorch\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                       last_epoch=-1)

with torch.cuda.device(gpu_index):
    simclr = SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=100, log_every_n_steps=100)
    simclr.train(train_loader, shuffle=True)

  0%|          | 0/234 [00:00<?, ?it/s]C:\Users\kacpe\DataspellProjects\SimCLRPytorch\venv\lib\site-packages\torch\optim\lr_scheduler.py:807: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
100%|██████████| 234/234 [00:29<00:00,  7.86it/s]


In [10]:
torch.save(model.state_dict(), 'saved_models/model_final.pth')

In [17]:
model

ConvModel(
  (resnet_model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

In [12]:
model.backbone.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=512, out_features=128, bias=True),
    torch.nn.ReLU(inplace=True),
    # torch.nn.BatchNorm2d(128,)
    torch.nn.Linear(in_features=128, out_features=10, bias=True),
)
# liniowa 512 -> 128
# relu
# batchnorm
# liniowa 128 -> 10

In [13]:
model.backbone.requires_grad_(False)
model.backbone.fc.requires_grad_(True)
model

ConvModel(
  (resnet_model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

In [14]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

with torch.cuda.device(gpu_index):
    simclr = SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=100, log_every_n_steps=100)
    simclr.train(val_loader, shuffle=False)


  0%|          | 0/235 [00:00<?, ?it/s]C:\Users\kacpe\DataspellProjects\SimCLRPytorch\venv\lib\site-packages\torch\optim\lr_scheduler.py:807: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
100%|██████████| 235/235 [00:07<00:00, 29.85it/s]
